In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import glob

from importlib import reload
import cartopy.io.shapereader as shapereader


In [5]:
import preseason as p


In [3]:
import preseason.seasonalityfunctions as sf
import preseason.onset_demise as od

ModuleNotFoundError: No module named 'preseason'

In [ ]:
reload(od)
reload(sf)

In [ ]:
### Peru Domain ###
#min_lon = -83+360
#min_lat = -20.0
#max_lon = -67+360
#max_lat = 0.0



### processing CPC data 


cpc_files = glob.glob('/Users/gbromley/data/cpc_uni_global/precip.*.nc')
cpc_raw = xr.open_mfdataset(cpc_files)

### Setting coordinate data for next step 
lat = np.arange(-90, 90, 0.5)
lon = np.arange(0, 360, 0.5)
time = cpc_raw['time'].data

### transposing dimensions so the world is right side up 

flipped_cpc = np.flip(cpc_raw['precip'].values, axis=1)
### cpc latitudes are reversed so that they are decreasing instead of increasing. The following code reverses it. 
cpc_pr = xr.DataArray(flipped_cpc, coords=[('time', time), ('lat', lat), ('lon', lon)], name='precipitation')

### Selecting out Peru from the global data.
PERU = {'lat': slice(-20,0), 'lon': slice(360-83, 360-67)}
cpc_conus = cpc_pr.sel(lat = PERU['lat'], lon = PERU['lon'])

In [ ]:
cpc_conus_anom = cpc_conus - cpc_conus.mean(dim='time')

In [ ]:
sf.fourier_coefficients(sf.calc_annual_cycle(cpc_conus).values)

In [ ]:
bom_files = glob.glob('/Users/gbromley/data/b17_cpc_onset-demise/onset.wet.season.CPC_UNI.*.nc')
test_bombardi = xr.open_mfdataset(sorted(bom_files), decode_cf = False, combine = 'nested', concat_dim='time')
test_bombardi = test_bombardi['DOY'].sel(PERU).squeeze()
test_bombardi = xr.where(test_bombardi == -999., np.nan, test_bombardi)

In [ ]:
pd.date_range('1979', freq='YS', periods = 42)

In [ ]:
test_bombardi['time'] = pd.date_range('1979', freq='YS', periods = 42)

In [ ]:
test_bombardi

In [ ]:
annual_cycle = sf.calc_annual_cycle(cpc_conus)

In [ ]:
start_wet = od.B17_analysis_start(annual_cycle)

In [ ]:
sf.

In [ ]:
(start_wet - lm_onset.isel(year = 20)).plot()

In [ ]:
reload(od)
time_slice = slice(4000, 4500)
test_loc = 20
temp = cpc_conus_anom.isel(lat=test_loc, lon=test_loc, time=time_slice)
temp_start = start_wet.isel(lat=test_loc, lon=test_loc)
test_data = od._onset_LM01(temp.values, time=temp.time,startWet=temp_start.values)
test_demise = od._demise_LM01(temp.values, time=temp.time,startWet=temp_start.values)

In [ ]:
plt.gca().get_xticklabels()

In [ ]:
import matplotlib.dates as mdates
import datetime
from matplotlib.legend_handler import HandlerLine2D

In [ ]:
ax1.get_xlim()

In [ ]:
num_years = 2
pre_color = '#30638E'
#
# '#3FA7D6'
#
cumul_color = '#193C16'
#'#F18F01'
#'#0D1431'
plt.rcParams['font.size'] = 24

fs = plt.rcParams['font.size']
#orange = F18F01
# Create a figure and a set of subplots
fig, ax1 = plt.subplots()
fig.set_size_inches((22,12))
# Plot the first data set with the first y-axis
anom_line = ax1.plot(temp.time, temp, pre_color, label=r'P$_{anom}$', alpha=0.8)  # 'g-' is for green solid line
ax1.set_xlabel('Day of Year')
ax1.set_ylabel(r'P$_{anom}$ (mm)', color=pre_color, fontsize=fs+4)  # Set the color of the y-axis label to green
ax1.tick_params(axis='y', labelcolor='black')

ax1.set_ylim(-7,25)

ax1.set_xlim(7300,7750)

ax1.fill_between(temp.time, temp, 0, alpha=0.5, where=temp > 0, facecolor = '#30638E')
ax1.fill_between(temp.time, temp, 0, alpha=0.5, where=temp < 0, facecolor='#857E61')

# Create a second y-axis that shares the same x-axis
ax2 = ax1.twinx()  
cumul = ax2.plot(temp.time, filtered_cumul, cumul_color, label=r'Cumul. $P_{anom}$', linewidth=3, alpha = 0.9)  # 'b-' is for blue solid line
ax2.set_ylabel(r'Accumulated P$_{anom}$ (mm)', color=cumul_color, fontsize=fs+4)  # Set the color of the y-axis label to blue
ax2.tick_params(axis='y', labelcolor='black')

ax2.set_ylim(-200, 800)
# Format date
#plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%j'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))

current_ticks = plt.gca().get_xticks()[1:]
current_labels = [mdates.num2date(tick).strftime('%-j') for tick in current_ticks]

#annotated ticks
start_year = str(np.unique(temp.time.dt.year.values)[1])
years = pd.date_range(start = start_year, periods=num_years, freq='YS')

onset_doy = test_data[1:]
demise_doy = test_demise[1:]

onset_dates = np.empty_like(onset_doy, dtype=np.dtype('M8[D]'))
onset_dates[:] = np.datetime64("NaT")

demise_dates = np.empty_like(demise_doy, dtype=np.dtype('M8[D]'))
demise_dates[:] = np.datetime64("NaT")


for i,j in enumerate(onset_doy):
    
    if not np.isnan(j):
        
        onset_dates[i] = years[i] + pd.Timedelta(int(j), unit='day')
        
for i,j in enumerate(demise_doy):
    if not np.isnan(j):
        
        demise_dates[i] = years[i] + pd.Timedelta(int(j), unit='day')

ticks_str = years

ticks_to_add = [int(mdates.date2num(item)) for item in ticks_str]

#TODO clean all odf this up.
additional_labels = [mdates.num2date(year).strftime('%b-%-d-%y') for year in ticks_to_add]

# Combine current and additional ticks and labels
all_ticks = np.concatenate((current_ticks, ticks_to_add))
all_labels = current_labels + additional_labels



for i in onset_dates:
    if not np.isnan(i):
        onset_line = plt.axvline(x=i, ymin=0, ymax=1, color='#6AA078', linewidth=2, linestyle='--')

for j in demise_dates:
    if not np.isnan(j):
        demise_line = plt.axvline(x=j, ymin=0, ymax=1, color='#C5C392', linewidth=2, linestyle='--')


onset_line.set_label(r'Onset$_{ wet}$')
demise_line.set_label(r'Onset$_{ dry}$')
# Rotate date labels

legend_artists = [anom_line[0], demise_line, onset_line, cumul[0]]
legend_labels = [r'P$_{anom}$', r'Onset$_{ dry}$', r'Onset$_{ wet}$',r'Cumul. $P_{anom}$']
fig.legend(legend_artists, legend_labels, loc='lower left', bbox_to_anchor=(.1175, 0.81, 0.79, .25), ncols=4, mode='expand', frameon=True)

#ax1.set_title('Location: '+str(np.abs(temp.lat.values))+' S, '+str(temp.lon.values)+' W'+' (Peru)', x=0.5, y=1.05)

#ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%b'))
plt.gca().set_xticks(all_ticks)
plt.gca().set_xticklabels(all_labels)
plt.gcf().autofmt_xdate(rotation=30)


# Set the combined ticks and labels

#plt.plot(temp.time, filtered_cumul)
#(temp*10).plot()
out_path = '/Users/gbromley/Dropbox/OU/Peru/'
plt.savefig(out_path+'Peru_Example.png', dpi=600, transparent=True)
plt.show()

In [ ]:
all_labels[0].set_rotation(30)

In [ ]:
all_labels

In [ ]:
ax1.set_xlim(-7,20)

In [ ]:
map_proj

In [ ]:
temp

In [ ]:
import cartopy.crs as ccrs
import cartopy
from matplotlib.offsetbox import AnchoredText

map_proj = ccrs.Orthographic(central_longitude = -75, central_latitude= -10)



fig, ax = plt.subplots(1, 1, figsize=(10, 13), dpi=600,  subplot_kw={'projection': map_proj})

ax.pcolormesh(cpc_conus.isel(time=100).values, alpha=0.5, transform=ccrs.PlateCarree())

ax.set_global()

ax.plot(285,-10,'bo', markersize=7)


In [ ]:
mdates.date2num(temp.time[0])

In [ ]:
current_ticks

In [ ]:
cpc_conus

In [ ]:
reload(od)
reload(sf)
bom_demise = od.demise_B17(cpc_conus,start_wet)



In [ ]:
1990-1979

In [ ]:
bom_onset['year'] 

In [ ]:
bom_onset.isel(year=11).plot()

In [ ]:
lm_onset = od.onset_LM01(cpc_conus, start_wet)

In [ ]:
cpc_conus.plot.countour

In [ ]:
plot_spatial_data(bom_onset.isel(year=11), title='Onset DOY for 1990', var='Day of Year')

In [ ]:
import matplotlib.ticker as mticker


from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

def plot_spatial_data(dataarray, projection=ccrs.PlateCarree(), title='Spatial Data Plot', var='data_to_plot'):
    """
    Plots a spatial figure of a variable from an xarray DataArray.

    :param dataarray: xarray DataArray containing the geospatial data to be plotted.
    :param projection: Cartopy CRS projection. Defaults to PlateCarree.
    :param title: Title of the plot.
    """
    
    states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m')
    map_proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=45)
    #cmap = mpl.cm.RdBu_r


    fig, ax = plt.subplots(1, 1, figsize=(10, 13), dpi=600,  subplot_kw={'projection': projection})
    p = dataarray.plot(ax=ax,transform=ccrs.PlateCarree(), add_colorbar=False, cmap='twilight',alpha = 0.8)


    ### Setting 1st plot parameters ###
    ax.coastlines(edgecolor='black', linewidth=2)
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black', linewidth=2)
    ax.add_feature(cfeature.STATES, edgecolor='black', linewidth=2)
    ax.add_feature(cfeature.LAKES, alpha=0.5, edgecolor='blue')
    ax.add_feature(cfeature.RIVERS, color='blue')
    #ax1.set_xticks(np.arange(-180,181, 40))
    #ax1.set_yticks(np.arange(-90,91,15))
    drainage = shapereader.Reader('/Users/gbromley/Downloads/major_basins_of_the_world_0_0_0/Major_Basins_of_the_World.shp')
    #for feature in drainage.records():
    #    geometry = feature.geometry
    #    ax.add_geometries([geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=2, linestyle='--')
    #at = AnchoredText("a",
    #                    loc='upper left', prop=dict(size=8), frameon=True,)
    #at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
    #ax1.add_artist(at)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)
    plt.colorbar(p, cax=cax, label=var)
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, linewidth=2, color='black', alpha=0.5, linestyle='--')
    gl.xlocator = mticker.FixedLocator([280-360, 285-360, 290-360])
    gl.ylocator = mticker.FixedLocator([-5, -10, -15])
    gl.left_labels = True
    gl.right_labels = False
    gl.top_labels = False
    gl.bottom_labels = True

    # Add a title
    ax.set_title(title, loc='center')

    # Show the plot
    plt.show()

In [ ]:
lm_onset.isel(year=11).plot()

In [ ]:
tseries = np.nan_to_num(test_data)
    
smoothie = np.copy(tseries)
temp = np.copy(tseries)
    
for n in np.arange(0,1):
    temp[0] = 0.5*(smoothie[0]+smoothie[1])
    temp[-1] = 0.5*(smoothie[-1]+smoothie[-2])
    temp[1:-1] = 0.25*smoothie[0:-2] + 0.5*smoothie[1:-1]+0.25*smoothie[2:]
    smoothie = temp

In [ ]:
0.25*smoothie[0:-2] + 0.5*smoothie[1:-1]+0.25*smoothie[2:]


In [ ]:
.25 * smoothie[0:-2]

In [ ]:
0.5*smoothie[1:-1]

In [ ]:
(0.25*smoothie[2:]) + (0.5*smoothie[1:-1])

In [ ]:
temp

In [ ]:
smoothie = np.copy(tseries)
0.25*smoothie[0:-2] + 0.5*smoothie[1:-1] + 0.25*smoothie[2:]



In [ ]:
temp

In [ ]:
reload(sf)
test_data = np.arange(0, 22.5, 1)
#test_data[0] = np.nan
#print(test_data)
nans = np.empty_like(test_data)
nans[:] = np.nan    

output = sf.smooth_B17(test_data)

#check =  not np.all(np.isnan(output))

In [ ]:
len(output)

In [ ]:
arr = np.random.rand(20)

    # Choose 3 random indices to replace with np.nan
    # np.random.choice generates a random sample from the given array
nan_indices = np.random.choice(20, 3, replace=False)

    # Replace the chosen indices with np.nan
arr[nan_indices] = np.nan

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import datetime

# Sample data
dates = [datetime.datetime(2020, month, 1) for month in range(1, 13)]
values = np.random.rand(len(dates))

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(dates, values)

# Usual date formatting for existing ticks
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

# Get current ticks and labels
current_ticks = plt.gca().get_xticks()
current_labels = [mdates.num2date(tick).strftime('%b %Y') for tick in current_ticks]

# Define additional custom ticks and their labels
#additional_ticks = [mdates.date2num(datetime.datetime(2020, 6, 15)), mdates.date2num(datetime.datetime(2020, 11, 15))]
additional_labels = [mdates.num2date(ticks).strftime('%b %Y') for ticks in additional_ticks]

# Combine current and additional ticks and labels
all_ticks = np.concatenate((current_ticks, additional_ticks))
all_labels = current_labels + additional_labels

# Set the combined ticks and labels
plt.gca().set_xticks(all_ticks)
plt.gca().set_xticklabels(all_labels)

# Rotate date labels for better readability
plt.gcf().autofmt_xdate()

plt.show()
